In [16]:
import csv
import collections
import numpy as np

In [3]:
train_data_dict = []
test_data_dict = []
with open('train_data.csv', 'r') as f:
    reader = csv.DictReader(f)
    for line in enumerate(reader):
        train_data_dict.append(line[1])
        
with open('test_data.csv', 'r') as f:
    reader = csv.DictReader(f)
    for line in enumerate(reader):
        test_data_dict.append(line[1])

In [31]:
grouped_by_game = collections.defaultdict(list)
for line in test_data_dict:
    grouped_by_game[line['gameid']].append(line)
grouped_by_game = list(grouped_by_game.values())

In [32]:
for game in grouped_by_game:
    for play_dict in game:
        for key in play_dict:
            try:
                play_dict[key] = int(play_dict[key])
            except:
                pass

In [33]:
for game in grouped_by_game:
    game.sort(key = lambda x: -int(x['time']))

In [34]:
train = []
y = []

for game in grouped_by_game:
    #game = grouped_by_game[key]
    
    num_plays_first = 0
    num_plays_second = 0
    num_plays_third = 0
    num_plays_fourth = 0
    score_diff = 0
    average_yards = 0
    
    for play_dict in game:
        if int(play_dict['time'])<=900:
            num_plays_fourth+=1
        elif int(play_dict['time'])<=1800:
            num_plays_third+=1
        elif int(play_dict['time'])<=2700:
            num_plays_second+=1
        else:
            num_plays_first+=1
        
        if int(play_dict['time']) >=900:
            score_diff = abs(int(play_dict['offscore'])-int(play_dict['defscore']))
        
    first_team_yards=0
    first_team_plays=0
    second_team_yards=0
    second_team_plays=0
    time_per_play = []
    for i in range(len(game)-1):
        play_dict = game[i]
        alph_first_team = play_dict['off'] if play_dict['off']<play_dict['def'] else play_dict['def']
        next_play_dict = game[i+1]
        if next_play_dict['down'] is not None and play_dict['down'] is not None and play_dict['ydline'] !='' and next_play_dict['ydline'] !='' and next_play_dict['off']==play_dict['off'] and next_play_dict['description'].find('punt')==-1:
            progress = play_dict['ydline']-next_play_dict['ydline']
            if play_dict['off']==alph_first_team:
                first_team_yards+=progress
                first_team_plays+=1
            else:
                second_team_yards+=progress
                second_team_plays+=1
            time_per_play.append(int(play_dict['time'])-int(next_play_dict['time']))
            
    first_team_avg_yards = first_team_yards/first_team_plays
    second_team_avg_yards = second_team_yards/second_team_plays
    total_avg_yards = (first_team_yards+second_team_yards)/(first_team_plays+second_team_plays)
    train.append((num_plays_first, num_plays_second, num_plays_third, score_diff, first_team_avg_yards, second_team_avg_yards, total_avg_yards, np.var(time_per_play), num_plays_fourth))
    y.append(num_plays_fourth)

In [35]:
train[0]

(42,
 51,
 40,
 22,
 5.160714285714286,
 4.397058823529412,
 4.741935483870968,
 253.48829344432886,
 38)

In [36]:
features = ['num_plays_first', 'num_plays_second', 'num_plays_third', 'score_diff', 'first_team_avg_yards', 'second_team_avg_yards', 'total_avg_yards', 'variance_time_per_play', 'num_plays_fourth']

In [37]:
with open('numplaysfeatures_test.csv','w') as out:
    writer = csv.writer(out)
    writer.writerow(features)
    for datum in train:
        writer.writerow(datum)